# Analyse exploratoire
## Détections d'anomalies
### Chargement du fichier

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

df_conso_enrichies = pd.read_parquet("../output/consommations_enrichies.parquet")

df_conso_enrichies.info()
df_conso_enrichies.head()
df_conso_enrichies['date'] = pd.to_datetime(df_conso_enrichies['date'])

df_conso_enrichies.head(10)

<class 'pandas.DataFrame'>
RangeIndex: 10620788 entries, 0 to 10620787
Data columns (total 32 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   type_energie             str           
 1   batiment_id              str           
 2   unite                    str           
 3   hour                     int32         
 4   year                     int32         
 5   month                    int32         
 6   conso_mean               float64       
 7   conso_min                float64       
 8   conso_max                float64       
 9   measurement_count        int64         
 10  date                     object        
 11  timestamp_hour           datetime64[ns]
 12  surface_m2               int32         
 13  nb_occupants_moyen       int32         
 14  type                     str           
 15  nom                      str           
 16  commune                  str           
 17  temperature_c            float64    

,type_energie,batiment_id,unite,hour,year,month,conso_mean,conso_min,conso_max,measurement_count,...,day_of_week,season,date_debut,date_fin,tarif_unitaire,cout_consommation,conso_par_occupant,conso_par_m2,ipe,ecart_a_la_moyenne
0,eau,BAT0042,m3,10,2023,1,16.25,16.25,16.25,1,...,0.0,Hiver,2024-01-01,2024-12-31,3.75,60.94,0.1806,0.0097,0.0097,-93.43
1,gaz,BAT0033,kWh,7,2024,8,49.78,49.78,49.78,1,...,NaN,NaN,2024-07-01,2024-12-31,0.12,5.97,1.2445,0.0753,0.0753,-59.90
2,eau,BAT0042,m3,10,2023,1,16.25,16.25,16.25,1,...,0.0,Hiver,2023-01-01,2023-12-31,3.50,56.88,0.1806,0.0097,0.0097,-93.43
3,gaz,BAT0033,kWh,7,2024,8,49.78,49.78,49.78,1,...,NaN,NaN,2024-01-01,2024-06-30,0.11,5.48,1.2445,0.0753,0.0753,-59.90
4,gaz,BAT0042,kWh,10,2023,1,624.68,624.68,624.68,1,...,0.0,Hiver,2024-07-01,2024-12-31,0.12,74.96,6.9409,0.3718,0.3718,515.00
5,gaz,BAT0033,kWh,7,2024,8,49.78,49.78,49.78,1,...,NaN,NaN,2023-07-01,2023-12-31,0.10,4.98,1.2445,0.0753,0.0753,-59.90
6,gaz,BAT0042,kWh,10,2023,1,624.68,624.68,624.68,1,...,0.0,Hiver,2024-01-01,2024-06-30,0.11,68.71,6.9409,0.3718,0.3718,515.00
7,gaz,BAT0033,kWh,7,2024,8,49.78,49.78,49.78,1,...,NaN,NaN,2023-01-01,2023-06-30,0.09,4.48,1.2445,0.0753,0.0753,-59.90
8,gaz,BAT0042,kWh,10,2023,1,624.68,624.68,624.68,1,...,0.0,Hiver,2023-07-01,2023-12-31,0.10,62.47,6.9409,0.3718,0.3718,515.00
9,gaz,BAT0036,kWh,7,2024,8,325.87,325.87,325.87,1,...,NaN,NaN,2024-07-01,2024-12-31,0.12,39.10,2.7616,0.1634,0.1634,216.19


### Pics de consommation anormaux (>3 écart type)

In [3]:
stats_anomalies = df_conso_enrichies.groupby('type_energie')['conso_mean'].agg(['mean', 'std']).reset_index()

df_check = df_conso_enrichies.merge(stats_anomalies, on='type_energie')

df_check['is_anomalie'] = df_check['conso_mean'] > (df_check['mean'] + 3 * df_check['std'])

df_pics = df_check[df_check['is_anomalie'] == True].copy()

# Sélection des colonnes stratégiques pour y voir clair
colonnes_cles = ['timestamp_hour', 'nom', 'type_energie', 'conso_mean', 'mean', 'std', 'commune']

# Affichage des 10 plus grosses anomalies
print("Top 10 des plus gros pics de consommation détectés :")
display(df_pics[colonnes_cles].sort_values(by='conso_mean', ascending=False).head(10))

Top 10 des plus gros pics de consommation détectés :


,timestamp_hour,nom,type_energie,conso_mean,mean,std,commune
3628824,2024-12-25 09:00:00,Piscine Paris 5,gaz,6348.19,406.312361,640.952003,Paris
3628822,2024-12-25 09:00:00,Piscine Paris 5,gaz,6348.19,406.312361,640.952003,Paris
3628820,2024-12-25 09:00:00,Piscine Paris 5,gaz,6348.19,406.312361,640.952003,Paris
3628818,2024-12-25 09:00:00,Piscine Paris 5,gaz,6348.19,406.312361,640.952003,Paris
3109924,2023-11-03 16:00:00,Piscine Paris 5,gaz,6345.10,406.312361,640.952003,Paris
3109922,2023-11-03 16:00:00,Piscine Paris 5,gaz,6345.10,406.312361,640.952003,Paris
3109918,2023-11-03 16:00:00,Piscine Paris 5,gaz,6345.10,406.312361,640.952003,Paris
3109920,2023-11-03 16:00:00,Piscine Paris 5,gaz,6345.10,406.312361,640.952003,Paris
2511032,2023-11-13 08:00:00,Piscine Paris 5,gaz,6343.90,406.312361,640.952003,Paris
2511034,2023-11-13 08:00:00,Piscine Paris 5,gaz,6343.90,406.312361,640.952003,Paris


### Périodes de sous-consommation suspectes

In [6]:
stats_moyenne = df_conso_enrichies.groupby('type_energie')['conso_mean'].mean().reset_index()
stats_moyenne.columns = ['type_energie', 'moyenne_energie']

df_basse_conso = df_conso_enrichies.merge(stats_moyenne, on='type_energie')

df_resultat = df_basse_conso[df_basse_conso['conso_mean'] < df_basse_conso['moyenne_energie']]

colonnes_cles = ['nom', 'type_energie', 'conso_mean', 'moyenne_energie', 'commune']

print("Les 10 consommations les plus basses (inférieures à la moyenne) :")
display(df_resultat[colonnes_cles].sort_values(by='conso_mean', ascending=True).head(10))

Les 10 consommations les plus basses (inférieures à la moyenne) :


,nom,type_energie,conso_mean,moyenne_energie,commune
9274047,Mairie Nice 96,eau,0.02,42.804353,Nice
9274049,Mairie Nice 96,eau,0.02,42.804353,Nice
9679439,Mairie Bordeaux 40,eau,0.02,42.804353,Bordeaux
10307181,Mairie Nice 96,eau,0.02,42.804353,Nice
10294684,Mairie Nice 96,eau,0.02,42.804353,Nice
8370323,Mairie Bordeaux 40,eau,0.02,42.804353,Bordeaux
8919842,Mairie Bordeaux 40,eau,0.02,42.804353,Bordeaux
9425116,Mairie Bordeaux 40,eau,0.02,42.804353,Bordeaux
9635434,Mairie Bordeaux 40,eau,0.02,42.804353,Bordeaux
9635436,Mairie Bordeaux 40,eau,0.02,42.804353,Bordeaux


### Bâtiments dont la consommation ne correspond pas à leur DPE

In [7]:
# "classe_energique" n'est pas dans mon dataframe, je la rajoute donc manuellement avec une jointure sur batiment_id 

df_bat_brut = pd.read_csv("../data_ecf/batiments.csv")

df_bat_light = df_bat_brut[['batiment_id', 'classe_energetique']] 

df_conso_enrichies = df_conso_enrichies.merge(df_bat_light, on='batiment_id', how='left')

print("La colonne classe_energetique a été réintégrée !")

La colonne classe_energetique a été réintégrée !


In [ ]:
stats_dpe = df_conso_enrichies.groupby('classe_energetique')['conso_mean'].mean().reset_index()
stats_dpe.columns = ['classe_energetique', 'conso_moyenne_classe']

df_dpe_check = df_conso_enrichies.merge(stats_dpe, on='classe_energetique')

df_dpe_check['ecart_classe'] = df_dpe_check['conso_mean'] - df_dpe_check['conso_moyenne_classe']

incoherences = df_dpe_check[df_dpe_check['ecart_classe'] > (df_dpe_check['conso_moyenne_classe'] * 0.5)]

cols_view = ['nom', 'classe_energetique', 'conso_mean', 'conso_moyenne_classe', 'ecart_classe', 'commune']

incoherences_choquantes = df_dpe_check[
    (df_dpe_check['classe_energetique'].isin(['A', 'B', 'C', 'D'])) & 
    (df_dpe_check['ecart_classe'] > (df_dpe_check['conso_moyenne_classe'] * 2))
]

print("Bâtiments sur-consommateurs par rapport à leur DPE :")
display(incoherences_choquantes[cols_view].sort_values(by='ecart_classe', ascending=False).head(10))


Bâtiments sur-consommateurs par rapport à leur DPE :


,nom,classe_energetique,conso_mean,conso_moyenne_classe,ecart_classe,commune
2722976,Piscine Marseille 24,D,3196.77,239.324231,2957.445769,Marseille
2722982,Piscine Marseille 24,D,3196.77,239.324231,2957.445769,Marseille
2722980,Piscine Marseille 24,D,3196.77,239.324231,2957.445769,Marseille
2722978,Piscine Marseille 24,D,3196.77,239.324231,2957.445769,Marseille
2906279,Piscine Marseille 24,D,3196.00,239.324231,2956.675769,Marseille
2906277,Piscine Marseille 24,D,3196.00,239.324231,2956.675769,Marseille
2906275,Piscine Marseille 24,D,3196.00,239.324231,2956.675769,Marseille
2906273,Piscine Marseille 24,D,3196.00,239.324231,2956.675769,Marseille
4172938,Piscine Marseille 24,D,3193.61,239.324231,2954.285769,Marseille
4172940,Piscine Marseille 24,D,3193.61,239.324231,2954.285769,Marseille


### Liste des bâtiments nécessitant un audit energetique

In [10]:
df_dpe_check['priorite_audit'] = df_dpe_check['conso_mean'] > (df_dpe_check['conso_moyenne_classe'] * 2)

liste_audit = df_dpe_check[df_dpe_check['priorite_audit'] == True].drop_duplicates(subset=['nom'])

def donner_conseil(classe):
    if classe in ['A', 'B', 'C']:
        return "Urgent : Suspicion d'erreur DPE ou réglages équipements"
    else:
        return "Rénovation thermique globale à prévoir"

liste_audit['recommandation'] = liste_audit['classe_energetique'].apply(donner_conseil)

cols_final = ['nom', 'commune', 'classe_energetique', 'conso_mean', 'recommandation']
display(liste_audit[cols_final].sort_values(by='conso_mean', ascending=False).head(15))

,nom,commune,classe_energetique,conso_mean,recommandation
2259687,Piscine Lille 50,Lille,F,3444.63,Rénovation thermique globale à prévoir
2259651,Piscine Lille 48,Lille,G,2955.97,Rénovation thermique globale à prévoir
2206494,Piscine Saint-Etienne 133,Saint-Etienne,F,2942.94,Rénovation thermique globale à prévoir
2311739,Piscine Montpellier 80,Montpellier,F,2881.35,Rénovation thermique globale à prévoir
2285595,Piscine Lyon 19,Lyon,F,2811.10,Rénovation thermique globale à prévoir
2184426,Piscine Reims 112,Reims,G,2373.59,Rénovation thermique globale à prévoir
2354463,Piscine Paris 5,Paris,G,2279.52,Rénovation thermique globale à prévoir
2188706,Piscine Rennes 99,Rennes,F,2264.47,Rénovation thermique globale à prévoir
2184427,Piscine Toulon 136,Toulon,F,2166.97,Rénovation thermique globale à prévoir
2214287,Piscine Le Havre 118,Le Havre,E,2166.44,Rénovation thermique globale à prévoir
